In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
from collections import defaultdict

# --------------------------------------------------------------
# 1. Generate synthetic social network (100 users, 4 topics)
# --------------------------------------------------------------
n_users = 100
n_topics = 4
users_per_topic = n_users // n_topics

topic_names = ["Sports", "Politics", "AI_Research", "Gaming"]
true_labels = [i // users_per_topic for i in range(n_users)]

p_within = 0.25
p_between = 0.02

G = nx.Graph()
G.add_nodes_from(range(n_users))

for i in range(n_users):
    for j in range(i+1, n_users):
        if true_labels[i] == true_labels[j]:
            if random.random() < p_within:
                G.add_edge(i, j)
        else:
            if random.random() < p_between:
                G.add_edge(i, j)

print(f"Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# --------------------------------------------------------------
# 2. Compute ALL centrality measures
# --------------------------------------------------------------

# 1. Degree Centrality
degree_centrality = nx.degree_centrality(G)

# 2. Betweenness Centrality (approximated for speed)
betweenness_centrality = nx.betweenness_centrality(G, k=100, seed=42)

# 3. Closeness Centrality
closeness_centrality = nx.closeness_centrality(G)

# 4. Eigenvector Centrality
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-6)

# 5. PageRank
pagerank = nx.pagerank(G, alpha=0.85)

# 6. NEW: Katz Centrality
# Choose α < 1/λ_max where λ_max is the largest eigenvalue of adjacency matrix
try:
    # Safest: use NetworkX built-in (automatically chooses α)
    katz_centrality = nx.katz_centrality(G, alpha=0.1, beta=1.0, max_iter=1000, tol=1e-6)
except:
    # Fallback: conservative α
    katz_centrality = nx.katz_centrality(G, alpha=0.05, beta=1.0)

# --------------------------------------------------------------
# 3. Get top 5 influencers for each measure
# --------------------------------------------------------------
def get_top_kols(centrality_dict, k=5):
    return sorted(centrality_dict.items(), key=lambda x: x[1], reverse=True)[:k]

top_katz = get_top_kols(katz_centrality)

centralities = {
    "Degree": get_top_kols(degree_centrality),
    "Betweenness": get_top_kols(betweenness_centrality),
    "Closeness": get_top_kols(closeness_centrality),
    "Eigenvector": get_top_kols(eigenvector_centrality),
    "PageRank": get_top_kols(pagerank),
    "Katz": top_katz
}

# --------------------------------------------------------------
# 4. Print results with topic labels
# --------------------------------------------------------------
print("\n" + "="*80)
print("TOP 5 INFLUENCERS (KOLs) BY CENTRALITY MEASURE")
print("="*80)

for name, tops in centralities.items():
    print(f"\n{name:12} Centrality → Top 5 Users:")
    for rank, (user, score) in enumerate(tops, 1):
        topic = topic_names[true_labels[user]]
        print(f"  #{rank} User {user:2d} ({topic:12}) → Score: {score:.4f}")

# --------------------------------------------------------------
# 5. Visualize: Node size by Katz Centrality
# --------------------------------------------------------------
plt.figure(figsize=(12, 9))
pos = nx.spring_layout(G, k=0.18, iterations=60, seed=42)

# Node sizes scaled by Katz centrality
katz_values = [katz_centrality[node] for node in G.nodes()]
node_sizes = [2000 * v + 100 for v in katz_values]  # scale up

# Color by true topic
node_colors = true_labels

nx.draw_networkx_edges(G, pos, alpha=0.15, width=0.5)
scatter = nx.draw_networkx_nodes(
    G, pos,
    node_size=node_sizes,
    node_color=node_colors,
    cmap=plt.cm.tab10,
    alpha=0.9
)

# Highlight top 3 Katz influencers with red border
top3_katz_users = [u for u, _ in top_katz[:3]]
nx.draw_networkx_nodes(
    G, pos,
    nodelist=top3_katz_users,
    node_size=800,
    node_color="red",
    edgecolors="black",
    linewidths=3,
    label="Top 3 Katz KOLs"
)

plt.title("100-User Social Network\nNode Size = Katz Centrality | Red Border = Top 3 KOLs", 
          fontsize=14, pad=20)
plt.axis("off")
plt.legend(scatterpoints=1)
plt.tight_layout()
plt.show()

# --------------------------------------------------------------
# 6. Bonus: Correlation between Katz and other measures
# --------------------------------------------------------------
import pandas as pd

df = pd.DataFrame({
    'Degree': degree_centrality,
    'Betweenness': betweenness_centrality,
    'Closeness': closeness_centrality,
    'Eigenvector': eigenvector_centrality,
    'PageRank': pagerank,
    'Katz': katz_centrality
})

print("\nCorrelation with Katz Centrality:")
print(df.corr()['Katz'].round(4).sort_values(ascending=False))

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

# ========================================
# 1. Create a polarized network with 100 users
# ========================================
n_left = 50   # Left community
n_right = 50  # Right community
n = n_left + n_right

G = nx.Graph()

# Add nodes with ideology attribute
for i in range(n_left):
    G.add_node(f"L{i}", ideology="Left")
for i in range(n_right):
    G.add_node(f"R{i}", ideology="Right")

# ========================================
# 2. Add dense connections WITHIN each side (homophily)
# ========================================
# Left community: high connection probability
for i in range(n_left):
    for j in range(i+1, n_left):
        if random.random() < 0.25:  # dense
            G.add_edge(f"L{i}", f"L{j}")

# Right community: high connection probability
for i in range(n_right):
    for j in range(i+1, n_right):
        if random.random() < 0.25:
            G.add_edge(f"R{i}", f"R{j}")

# ========================================
# 3. Add ONLY 4 cross-ideology bridges (the interesting part!)
# ========================================
bridges = [
    ("L5",  "R8"),   # a journalist both sides follow
    ("L12", "R3"),   # a moderate politician
    ("L23", "R19"),  # local news outlet
    ("L41", "R33")   # satire account
]

for a, b in bridges:
    G.add_edge(a, b)

# ========================================
# 4. Compute betweenness centrality
# ========================================
betweenness = nx.betweenness_centrality(G)

# Convert to sorted list
sorted_betweenness = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)

print("Top 15 nodes by betweenness centrality:")
print("-" * 50)
for node, bc in sorted_betweenness[:15]:
    ideology = G.nodes[node]['ideology']
    is_bridge = "BRIDGE" if any((node == a or node == b) for a, b in bridges) else ""
    print(f"{node:4} | Betweenness: {bc:6.4f} | {ideology:5} {is_bridge}")

# ========================================
# 5. Visualize the network
# ========================================
plt.figure(figsize=(12, 8))

pos = nx.spring_layout(G, seed=42)

# Color by ideology
node_colors = ['#ff4444' if G.nodes[n]['ideology'] == 'Left' else '#4444ff' for n in G.nodes()]

# Make bridge nodes larger and yellow
node_sizes = []
for node in G.nodes():
    if any((node == a or node == b) for a, b in bridges):
        node_sizes.append(400)
    else:
        node_sizes.append(80)

nx.draw(G, pos,
        node_color=node_colors,
        node_size=node_sizes,
        with_labels=False,
        edge_color='gray',
        alpha=0.7)

# Highlight bridges in yellow
bridge_nodes = set()
for a, b in bridges:
    bridge_nodes.add(a)
    bridge_nodes.add(b)

nx.draw_networkx_nodes(G, pos,
                       nodelist=bridge_nodes,
                       node_color='yellow',
                       node_size=500,
                       edgecolors='black',
                       linewidths=2)

plt.title("Polarized Network with 4 Cross-Ideology Bridges (yellow nodes)\n"
          "Betweenness centrality perfectly detects them!", fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------------
# 1. Simulate Customer Interaction Data
# ----------------------------
# Example: platform where users collaborate (message, share files, mention)
# Format: (user, interacts_with)
interactions = [
    ("user1", "user2"),
    ("user1", "user3"),
    ("user2", "user3"),
    ("user4", "user5"),
    ("user5", "user4"),
    # Isolated or near-isolated users (churn risk)
    ("user6", "user7"),  # only one weak link
    ("user8", None),     # completely isolated
    ("user9", None),
    ("user10", "user11"),
    ("user11", "user10"),
    ("user12", None),    # silent user
]

# Build an undirected graph of user interactions
G = nx.Graph()

# Add all users
all_users = set()
for u, v in interactions: # 取出senter, receiver
    all_users.add(u)
    if v is not None:
        all_users.add(v)

G.add_nodes_from(all_users)

# Add edges where interaction exists
for u, v in interactions:
    if v is not None:
        G.add_edge(u, v)

print(f"Total users: {G.number_of_nodes()}")
print(f"Connected users: {len([n for n in G.nodes() if G.degree(n) > 0])}")
print(f"Isolated users: {len([n for n in G.nodes() if G.degree(n) == 0])}")

# ----------------------------
# 2. Identify At-Risk (Churn-Prone) Customers
# ----------------------------
churn_risk = []

for user in G.nodes():
    degree = G.degree(user)
    # Ego network = user + direct neighbors
    ego = nx.ego_graph(G, user, radius=1)
    ego_size = ego.number_of_nodes()  # 1 = isolated
    
    # Optional: clustering coefficient (how connected are their friends?)
    # Not defined for degree < 2
    clustering = nx.clustering(G, user) if degree >= 2 else 0.0
    
    # Heuristic for churn risk:
    # - Degree == 0 → completely isolated
    # - Degree == 1 and ego_size == 2 → only one weak tie
    if degree == 0 or (degree == 1 and ego_size == 2):
        churn_risk.append({
            "user": user,
            "degree": degree,
            "ego_size": ego_size,
            "clustering": clustering
        })

print(f"\n {len(churn_risk)} users flagged for high churn risk:")
for r in churn_risk:
    print(f"  {r['user']}: degree={r['degree']}, ego_size={r['ego_size']}")

# ----------------------------
# 3. Prioritize for Retention Campaigns
# ----------------------------
# Example: Offer onboarding help, feature tutorial, or referral bonus
retention_actions = {
    "degree=0": "Send re-engagement email + product walkthrough",
    "degree=1": "Suggest connections or team members to invite"
}

print("\n Suggested retention actions:")
for r in churn_risk:
    if r["degree"] == 0:
        action = retention_actions["degree=0"]
    else:
        action = retention_actions["degree=1"]
    print(f"  {r['user']} → {action}")

# ----------------------------
# 4. Visualize Network (Highlight At-Risk Users)
# ----------------------------
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, seed=42)

# Color nodes: green = connected, red = at-risk
node_colors = []
node_sizes = []
for node in G.nodes():
    if node in [r["user"] for r in churn_risk]:
        node_colors.append("red")
        node_sizes.append(600)
    else:
        node_colors.append("lightgreen")
        node_sizes.append(400)

nx.draw(
    G, pos,
    node_color=node_colors,
    node_size=node_sizes,
    with_labels=True,
    font_size=10,
    edge_color="gray"
)

plt.title("Customer Interaction Graph\n Red = High Churn Risk (Isolated)")
plt.tight_layout()
plt.show()    

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# ----------------------------
# 1. Simulate a Referral/Interaction Network
# ----------------------------
# Format: (referrer, referred) — or (user1, user2) for mutual interaction
edges = [
    # Community A: Tech enthusiasts
    ("Alice", "Bob"),
    ("Bob", "Charlie"),
    ("Alice", "Charlie"),
    ("Charlie", "Diana"),
    
    # Community B: Fitness group
    ("Eve", "Frank"),
    ("Frank", "Grace"),
    ("Eve", "Grace"),
    
    # Community C: Sustainability advocates
    ("Helen", "Ivy"),
    ("Ivy", "Jack"),
    ("Helen", "Jack"),
    
    #  BRIDGE NODES (potential influencers)
    ("Diana", "Eve"),   # connects Tech ↔ Fitness
    ("Grace", "Helen"), # connects Fitness ↔ Sustainability
    ("Jack", "Alice"),  # connects Sustainability ↔ Tech → forms a cycle
]

# Build an undirected graph (referrals are mutual in influence)
G = nx.Graph()
G.add_edges_from(edges) # edge list

print(f"Network: {G.number_of_nodes()} users, {G.number_of_edges()} connections")

# ----------------------------
# 2. Compute Betweenness Centrality
# ----------------------------
betweenness = nx.betweenness_centrality(G)

# Sort by betweenness (descending)
influencers = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)

print("\n Top Influencers by Betweenness Centrality:")
print("User\t\tBetweenness\tRole")
print("-" * 45)
for user, score in influencers[:5]:
    role = "Bridge" if score > 0.2 else "Local"
    print(f"{user:<12}\t{score:.3f}\t\t{role}")

# ----------------------------
# 3. Why Betweenness > Degree?
# ----------------------------
degree = dict(G.degree())
print("\n Degree vs. Betweenness (Top 5 by degree):")
print("User\tDegree\tBetweenness")
print("-" * 30)
for user in sorted(degree, key=degree.get, reverse=True)[:5]:
    print(f"{user:<12}\t{degree[user]}\t{betweenness[user]:.3f}")

# → Notice: high-degree users may have LOW betweenness!

# ----------------------------
# 4. Visualize: Highlight Bridge Influencers
# ----------------------------
plt.figure(figsize=(12, 8))

# Get the current axes
ax = plt.gca()

# Layout
pos = nx.spring_layout(G, seed=42)

# Color nodes by betweenness (red = high)
node_colors = [betweenness[node] for node in G.nodes()]
node_sizes = [3000 * betweenness[node] + 300 for node in G.nodes()]

# Draw the graph
nx.draw(
    G, pos,
    node_color=node_colors,
    cmap=plt.cm.plasma,
    node_size=node_sizes,
    with_labels=True,
    font_size=10,
    edge_color="gray",
    width=1.5,
    ax=ax  # Explicitly pass ax
)

# Add colorbar using the same axes
sm = plt.cm.ScalarMappable(
    cmap=plt.cm.plasma,
    norm=plt.Normalize(vmin=min(node_colors), vmax=max(node_colors))
)
sm.set_array([])  # Required for older matplotlib versions
plt.colorbar(sm, ax=ax, label="Betweenness Centrality")  # <-- pass ax here

plt.title("Referral Network: Node Size & Color = Betweenness Centrality\n Red/Large = Key Influencers (Bridge Communities)")
plt.tight_layout()
plt.show()

# ----------------------------
# 5. Marketing Action: Target Bridge Influencers
# ----------------------------
top_influencer = influencers[0][0]
print(f"\n Recommendation: Partner with '{top_influencer}' for campaign.")
print("Why? They connect multiple communities—maximizing spread and diversity of reach.")

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform
from sklearn.datasets import make_spd_matrix

# ----------------------------
# 1. Simulate Stock Returns
# ----------------------------
np.random.seed(42)
n_assets = 10
n_days = 250

# Generate a realistic (positive semi-definite) covariance matrix
cov_matrix = make_spd_matrix(n_assets, random_state=42)
# Simulate daily returns from multivariate normal
returns = np.random.multivariate_normal(mean=np.zeros(n_assets), cov=cov_matrix, size=n_days)

# Create a DataFrame with asset labels
assets = [f"Asset_{i}" for i in range(1, n_assets + 1)]
returns_df = pd.DataFrame(returns, columns=assets)

# ----------------------------
# 2. Compute Correlation Matrix
# ----------------------------
corr = returns_df.corr()
print("Correlation matrix (top-left 5x5):")
print(corr.iloc[:5, :5].round(2))

# ----------------------------
# 3. Build Correlation Network
# ----------------------------
# Convert correlation to distance: d = sqrt(0.5 * (1 - corr))
dist_matrix = np.sqrt(0.5 * (1 - corr))

# Create a complete graph with distance as edge weight
G = nx.Graph()
for i in range(n_assets):
    for j in range(i + 1, n_assets):
        asset_i = assets[i]
        asset_j = assets[j]
        distance = dist_matrix.iloc[i, j]
        G.add_edge(asset_i, asset_j, weight=distance)

        # ----------------------------
# 4. Extract Minimum Spanning Tree (MST)
# ----------------------------
mst = nx.minimum_spanning_tree(G, weight='weight')

# ----------------------------
# 5. Analyze Risk: Centrality in MST
# ----------------------------
# In MST, high degree or betweenness = systemic risk hub
degree_centrality = nx.degree_centrality(mst)
betweenness = nx.betweenness_centrality(mst)

# Rank assets by betweenness (most influential in risk propagation)
risk_ranking = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)
print("\nTop risk-conduit assets (by betweenness centrality in MST):")
for asset, score in risk_ranking[:3]:
    print(f"  {asset}: {score:.3f}")
    
# ----------------------------
# 6. Visualize MST
# ----------------------------
plt.figure(figsize=(10, 6))
pos = nx.spring_layout(mst, seed=42)

# Node size proportional to betweenness
node_sizes = [5000 * betweenness[node] + 300 for node in mst.nodes()]

nx.draw(mst, pos,
        with_labels=True,
        node_size=node_sizes,
        node_color='lightgreen',
        font_size=9,
        edge_color='gray')

plt.title("Portfolio Risk Network: Minimum Spanning Tree\n(Node size ∝ risk influence)")
plt.tight_layout()
plt.show() 

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# ----------------------------
# 1. Build a Denser Network with 30 Firms
# ----------------------------
np.random.seed(42)
n_firms = 30
firm_names = [f"Firm_{i}" for i in range(n_firms)]

G = nx.DiGraph()
G.add_nodes_from(firm_names)

# Create a more interconnected network: each firm supplies 2–4 others on average
for i in range(n_firms):
    num_customers = np.random.randint(2, 5)  # more connections
    candidates = [f for f in firm_names if f != firm_names[i]]
    customers = np.random.choice(candidates, size=min(num_customers, len(candidates)), replace=False)
    for c in customers:
        G.add_edge(firm_names[i], c)

# Assign capacities
firm_data = {firm: {'capacity': np.random.uniform(100, 600)} for firm in firm_names}

# Assign supply shares (normalize per customer)
for customer in firm_names:
    suppliers = list(G.predecessors(customer))
    if suppliers:
        weights = np.random.dirichlet(np.ones(len(suppliers)) * 2)  # smoother distribution
        for i, supplier in enumerate(suppliers):
            G[supplier][customer]['supply_share'] = weights[i]

print(f"Network: {G.number_of_nodes()} firms, {G.number_of_edges()} supply links")

# ----------------------------
# 2. Compute Centrality Metrics
# ----------------------------
pagerank = nx.pagerank(G, weight='supply_share', alpha=0.85)
G_undir = G.to_undirected()
betweenness = nx.betweenness_centrality(G_undir, weight='supply_share')

# Find firm with highest combined centrality
combined_score = {f: pagerank[f] + betweenness[f] for f in firm_names}
disrupted_firm = max(combined_score, key=combined_score.get)

print(f"\nDisrupting: {disrupted_firm}")
print(f"  PageRank: {pagerank[disrupted_firm]:.4f}")
print(f"  Betweenness: {betweenness[disrupted_firm]:.4f}")

# ----------------------------
# 3. Simulate Disruption (Lower Threshold = More Cascade)
# ----------------------------
def simulate_disruption_cascade(G, firm_data, disrupted_firm, threshold=0.5, max_tiers=4):
    status = {f: 'operational' for f in firm_names}
    status[disrupted_firm] = 'disrupted'
    affected_by_tier = {0: {disrupted_firm}}
    newly_disrupted = {disrupted_firm}

    for tier in range(1, max_tiers + 1):
        next_disrupted = set()
        for firm in newly_disrupted:
            for customer in G.successors(firm):
                if status[customer] != 'operational':
                    continue
                total_loss = sum(
                    G[sup][customer].get('supply_share', 0)
                    for sup in G.predecessors(customer)
                    if status[sup] == 'disrupted'
                )
                if total_loss >= threshold:  # lowered from 0.7 → 0.5
                    status[customer] = 'disrupted'
                    next_disrupted.add(customer)
        if not next_disrupted:
            break
        affected_by_tier[tier] = next_disrupted
        newly_disrupted = next_disrupted

    total_output_loss = sum(firm_data[f]['capacity'] for f in firm_names if status[f] == 'disrupted')
    default_count = sum(1 for s in status.values() if s == 'disrupted')
    return affected_by_tier, total_output_loss, default_count, status

affected, output_loss, defaults, final_status = simulate_disruption_cascade(
    G, firm_data, disrupted_firm, threshold=0.5, max_tiers=4
)

# ----------------------------
# 4. Visualization
# ----------------------------
pos = nx.spring_layout(G, seed=42, k=2.0, iterations=100)

# Safely compute max_tier (avoid division by zero)
max_tier = max(1, max(affected.keys()))  # ensures at least 1

tier_color_map = {}
for tier, firms in affected.items():
    color = plt.cm.plasma(tier / max_tier)  # Now safe!
    for f in firms:
        tier_color_map[f] = color

node_colors = [tier_color_map.get(node, (0.85, 0.85, 0.85, 1.0)) for node in G.nodes()]
node_sizes = [200 + 3 * firm_data[node]['capacity'] / 10 for node in G.nodes()]

plt.figure(figsize=(14, 10))

# Draw edges
nx.draw_networkx_edges(
    G, pos,
    edge_color='lightgray',
    arrows=True,
    arrowsize=10,
    width=0.8,
    alpha=0.7
)

# Draw nodes
nx.draw_networkx_nodes(
    G, pos,
    node_color=node_colors,
    node_size=node_sizes,
    alpha=0.95
)

# Label ALL disrupted firms
disrupted_nodes = [n for n, s in final_status.items() if s == 'disrupted']
nx.draw_networkx_labels(
    G, pos,
    labels={n: n for n in disrupted_nodes},
    font_size=9,
    font_weight='bold',
    font_color='black'
)

# Legend
legend_elements = [
    Patch(facecolor=plt.cm.plasma(t / max_tier), label=f'Tier {t} (n={len(affected[t])})')
    for t in sorted(affected.keys())
]
legend_elements.append(Patch(facecolor=(0.85, 0.85, 0.85), label='Unaffected'))
plt.legend(handles=legend_elements, loc='upper right', title="Disruption Tier")

# Title
total_capacity = sum(f['capacity'] for f in firm_data.values())
loss_pct = (output_loss / total_capacity) * 100
plt.title(
    f"Supply Chain Disruption Cascade (30 Firms)\n"
    f"Initial failure: {disrupted_firm} | "
    f"{defaults}/{n_firms} firms disrupted ({loss_pct:.1f}% output loss)",
    fontsize=14,
    pad=20
)
plt.axis('off')
plt.tight_layout()
plt.show()

# ----------------------------
# 5. Print Summary
# ----------------------------
print(f"\n=== CASCADE SUMMARY ===")
print(f"Total firms: {n_firms}")
print(f"Disrupted firms: {defaults} ({100 * defaults / n_firms:.1f}%)")
print(f"Total output loss: {output_loss:.1f} units ({loss_pct:.1f}% of system capacity)")
print("\nAffected by tier:")
for tier in sorted(affected.keys()):
    print(f"  Tier {tier}: {list(affected[tier])}")

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import pandas as pd

# ----------------------------
# 1. Build Dense Supply Network (30 firms)
# ----------------------------
np.random.seed(123)
n_firms = 30
firm_names = [f"Firm_{i}" for i in range(n_firms)]

G = nx.DiGraph()
G.add_nodes_from(firm_names)

# Increase connectivity to ensure cascade
for i in range(n_firms):
    num_customers = np.random.randint(3, 6)  # Each firm supplies 3–5 others
    candidates = [f for f in firm_names if f != firm_names[i]]
    if candidates:
        customers = np.random.choice(candidates, size=min(num_customers, len(candidates)), replace=False)
        for c in customers:
            G.add_edge(firm_names[i], c)

# Assign daily production capacity (output)
firm_data = {firm: {'capacity': np.random.uniform(100, 600)} for firm in firm_names}

# Assign supply dependency shares
for customer in firm_names:
    suppliers = list(G.predecessors(customer))
    if suppliers:
        weights = np.random.dirichlet(np.ones(len(suppliers)) * 1.5)
        for i, supplier in enumerate(suppliers):
            G[supplier][customer]['supply_share'] = weights[i]

print(f"Network: {G.number_of_nodes()} firms, {G.number_of_edges()} supply links")

# ----------------------------
# 2. Compute Centrality & Choose Disruption Target
# ----------------------------
pagerank = nx.pagerank(G, weight='supply_share', alpha=0.85)
G_undir = G.to_undirected()
betweenness = nx.betweenness_centrality(G_undir, weight='supply_share')

# Pick firm with highest combined centrality
combined_score = {f: pagerank[f] + betweenness[f] for f in firm_names}
disrupted_firm = max(combined_score, key=combined_score.get)

print(f"\n🔥 Disrupting: {disrupted_firm}")
print(f"   PageRank: {pagerank[disrupted_firm]:.4f} | Betweenness: {betweenness[disrupted_firm]:.4f}")

# ----------------------------
# 3. Simulate Disruption Cascade
# ----------------------------
def simulate_disruption_cascade(G, firm_data, disrupted_firm, threshold=0.45, max_tiers=4):
    status = {f: 'operational' for f in firm_names}
    status[disrupted_firm] = 'disrupted'
    affected_by_tier = {0: {disrupted_firm}}
    newly_disrupted = {disrupted_firm}

    for tier in range(1, max_tiers + 1):
        next_disrupted = set()
        for firm in newly_disrupted:
            for customer in G.successors(firm):
                if status[customer] != 'operational':
                    continue
                total_loss = sum(
                    G[sup][customer].get('supply_share', 0)
                    for sup in G.predecessors(customer)
                    if status[sup] == 'disrupted'
                )
                if total_loss >= threshold:
                    status[customer] = 'disrupted'
                    next_disrupted.add(customer)
        if not next_disrupted:
            break
        affected_by_tier[tier] = next_disrupted
        newly_disrupted = next_disrupted

    total_output_loss = sum(firm_data[f]['capacity'] for f in firm_names if status[f] == 'disrupted')
    default_count = sum(1 for s in status.values() if s == 'disrupted')
    return affected_by_tier, total_output_loss, default_count, status

affected, total_loss, num_disrupted, final_status = simulate_disruption_cascade(
    G, firm_data, disrupted_firm, threshold=0.45, max_tiers=4
)

# ----------------------------
# 4. Print Detailed Disruption Report
# ----------------------------
print("\n" + "="*60)
print("💥 DISRUPTION IMPACT REPORT")
print("="*60)

# List all disrupted firms with their output loss
disrupted_list = []
for firm in firm_names:
    if final_status[firm] == 'disrupted':
        loss = firm_data[firm]['capacity']
        tier = next((t for t, firms in affected.items() if firm in firms), None)
        disrupted_list.append({
            'Firm': firm,
            'Tier': tier,
            'Daily Output Loss': loss
        })

# Create DataFrame for clean display
df = pd.DataFrame(disrupted_list)
df = df.sort_values(['Tier', 'Daily Output Loss'], ascending=[True, False])

print(df.to_string(index=False, float_format="{:.1f}".format))

print(f"\n📈 TOTAL SYSTEM IMPACT:")
print(f"   • Disrupted firms: {num_disrupted} / {n_firms} ({100 * num_disrupted / n_firms:.1f}%)")
print(f"   • Total daily output loss: {total_loss:.1f} units")
total_capacity = sum(f['capacity'] for f in firm_data.values())
print(f"   • System-wide capacity loss: {100 * total_loss / total_capacity:.1f}%")

# ----------------------------
# 5. Visualization
# ----------------------------
pos = nx.spring_layout(G, seed=42, k=2.5, iterations=100)

# Safe max_tier to avoid division by zero
max_tier = max(1, max(affected.keys()))

tier_color_map = {}
for tier, firms in affected.items():
    color = plt.cm.plasma(tier / max_tier)
    for f in firms:
        tier_color_map[f] = color

node_colors = [tier_color_map.get(node, (0.85, 0.85, 0.85, 1.0)) for node in G.nodes()]
node_sizes = [200 + 3 * firm_data[node]['capacity'] / 10 for node in G.nodes()]

plt.figure(figsize=(14, 10))

nx.draw_networkx_edges(G, pos, edge_color='lightgray', arrows=True, arrowsize=10, width=0.8, alpha=0.7)
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes, alpha=0.95)

# Label ALL disrupted firms
disrupted_nodes = [n for n, s in final_status.items() if s == 'disrupted']
nx.draw_networkx_labels(G, pos, labels={n: n for n in disrupted_nodes}, font_size=9, font_weight='bold')

# Legend
legend_elements = [
    Patch(facecolor=plt.cm.plasma(t / max_tier), label=f'Tier {t} (n={len(affected[t])})')
    for t in sorted(affected.keys())
]
legend_elements.append(Patch(facecolor=(0.85, 0.85, 0.85), label='Unaffected'))
plt.legend(handles=legend_elements, loc='upper right', title="Disruption Tier")

plt.title(
    f"Supply Chain Disruption: {disrupted_firm} Failed\n"
    f"{num_disrupted}/{n_firms} firms disrupted | {100 * total_loss / total_capacity:.1f}% output loss",
    fontsize=14, pad=20
)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# 1. Build a sample supply chain network
G = nx.DiGraph()
G.add_edges_from([
    ("Supplier_X", "Manufacturer_A"),
    ("Supplier_Y", "Manufacturer_A"),
    ("Supplier_Z", "Manufacturer_B"),
    ("Manufacturer_A", "Distributor_1"),
    ("Manufacturer_B", "Distributor_1"),
    ("Distributor_1", "Retailer_Final"),
])

# 2. Compute centrality metrics
pagerank = nx.pagerank(G, alpha=0.85)
betweenness = nx.betweenness_centrality(G.to_undirected())  # Betweenness uses undirected paths

# 3. Identify high-risk nodes (combine PageRank and Betweenness)
critical_firms = sorted(
    [(node, pr, betweenness[node]) for node, pr in pagerank.items()],
    key=lambda x: (x[1] + x[2]),
    reverse=True
)

print("Critical firms (High PageRank + High Betweenness):")
for firm, pr, bt in critical_firms[:3]:
    print(f"  {firm}: PageRank={pr:.3f}, Betweenness={bt:.3f}")

# Select the most critical firm to disrupt
most_critical = critical_firms[0][0]
print(f"\n→ Simulating disruption of: {most_critical}")

# 4. Simulate disruption: remove the critical node
G_disrupted = G.copy()
if most_critical in G_disrupted:
    G_disrupted.remove_node(most_critical)

# Count affected downstream customers (nodes that lost at least one supplier)
original_customers = set(n for n in G.nodes() if G.in_degree(n) > 0)
remaining_customers = set(n for n in G_disrupted.nodes() if G_disrupted.in_degree(n) > 0)
affected_count = len(original_customers - remaining_customers)

print(f"\nAfter removing {most_critical}, number of affected customers: {affected_count}")

# ----------------------------
# 5. VISUALIZATION
# ----------------------------
plt.figure(figsize=(14, 6))

# --- Left: Original Network ---
plt.subplot(1, 2, 1)
pos = nx.spring_layout(G, seed=42)

# Color the most critical node in red
node_colors_orig = ['red' if node == most_critical else 'lightblue' for node in G.nodes()]
nx.draw(
    G, pos,
    node_color=node_colors_orig,
    node_size=1200,
    with_labels=True,
    font_size=10,
    font_weight='bold',
    arrows=True,
    arrowsize=15,
    edge_color='gray'
)
plt.title(f"Original Supply Chain\n(Red = Most Critical Node: {most_critical})")

# --- Right: Disrupted Network ---
plt.subplot(1, 2, 2)
# Keep same layout positions for comparable view (remove missing nodes)
pos_disrupted = {node: pos[node] for node in G_disrupted.nodes() if node in pos}

node_colors_disrupted = ['lightcoral' if G_disrupted.in_degree(node) == 0 and node != most_critical else 'lightblue'
                         for node in G_disrupted.nodes()]

nx.draw(
    G_disrupted, pos_disrupted,
    node_color=node_colors_disrupted,
    node_size=1200,
    with_labels=True,
    font_size=10,
    font_weight='bold',
    arrows=True,
    arrowsize=15,
    edge_color='gray'
)

# Highlight orphaned customers (lost all suppliers)
orphaned = [n for n in G_disrupted.nodes() if G_disrupted.in_degree(n) == 0]
if orphaned:
    nx.draw_networkx_nodes(
        G_disrupted, pos_disrupted,
        nodelist=orphaned,
        node_color='orange',
        node_size=1200
    )

plt.title(f"After Removing {most_critical}\n(Orange = Orphaned Customers)")

plt.tight_layout()
plt.show()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# ----------------------------
# 1. Build a Larger, Layered Supply Chain Network
# ----------------------------
G = nx.DiGraph()

# Define layers
suppliers = [f"Supplier_{i}" for i in range(1, 6)]          # 5 suppliers
manufacturers = [f"Manufacturer_{i}" for i in range(1, 5)]   # 4 manufacturers
distributors = [f"Distributor_{i}" for i in range(1, 4)]     # 3 distributors
retailers = [f"Retailer_{i}" for i in range(1, 6)]          # 5 retailers

# Add all nodes
all_nodes = suppliers + manufacturers + distributors + retailers
G.add_nodes_from(all_nodes)

# Connect suppliers → manufacturers (each manufacturer uses 2–3 suppliers)
import random
random.seed(42)
for m in manufacturers:
    chosen_suppliers = random.sample(suppliers, k=random.randint(2, 3))
    for s in chosen_suppliers:
        G.add_edge(s, m)

# Connect manufacturers → distributors (each distributor gets from 2–3 manufacturers)
for d in distributors:
    chosen_manufacturers = random.sample(manufacturers, k=random.randint(2, 3))
    for m in chosen_manufacturers:
        G.add_edge(m, d)

# Connect distributors → retailers (each retailer served by 1–2 distributors)
for r in retailers:
    chosen_distributors = random.sample(distributors, k=random.randint(1, 2))
    for d in chosen_distributors:
        G.add_edge(d, r)

print(f"Network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# ----------------------------
# 2. Compute Centrality Metrics
# ----------------------------
pagerank = nx.pagerank(G, alpha=0.85)
betweenness = nx.betweenness_centrality(G.to_undirected())

# Combine scores to find most critical node
combined_scores = {node: pagerank[node] + betweenness[node] for node in G.nodes()}
most_critical = max(combined_scores, key=combined_scores.get)

print(f"\nMost critical node: {most_critical}")
print(f"  PageRank: {pagerank[most_critical]:.4f}")
print(f"  Betweenness: {betweenness[most_critical]:.4f}")

# ----------------------------
# 3. Simulate Disruption
# ----------------------------
G_disrupted = G.copy()
G_disrupted.remove_node(most_critical)

# Count customers who lost ALL their suppliers (in-degree = 0) after disruption
orphaned_after = [
    n for n in G_disrupted.nodes()
    if G_disrupted.in_degree(n) == 0 and n != most_critical
]

print(f"\nAfter removing {most_critical}:")
print(f"  Orphaned nodes (no incoming supply): {len(orphaned_after)}")
if orphaned_after:
    print("  Affected nodes:", ", ".join(orphaned_after))

# ----------------------------
# 4. Visualization
# ----------------------------
# Assign positions by layer for clarity
# Assign positions by layer for clarity
pos = {}
layer_spacing = 3

# Each entry: (list_of_nodes, y_level)
layers = [
    (suppliers, 0),
    (manufacturers, 1),
    (distributors, 2),
    (retailers, 3)
]

for node_list, y_level in layers:
    n = len(node_list)
    # Center the group horizontally
    x_positions = [i * layer_spacing for i in range(n)]
    x_center = (n - 1) * layer_spacing / 2
    for i, node in enumerate(node_list):
        pos[node] = (x_positions[i] - x_center, y_level)
        
# --- Plot Original vs Disrupted ---
plt.figure(figsize=(16, 8))

# Original Network
plt.subplot(1, 2, 1)
node_colors_orig = ['red' if node == most_critical else 'lightblue' for node in G.nodes()]
nx.draw(
    G, pos,
    node_color=node_colors_orig,
    node_size=900,
    with_labels=True,
    font_size=8,
    font_weight='bold',
    arrows=True,
    arrowsize=12,
    edge_color='gray',
    width=1.0
)
plt.title(f"Original Supply Chain\n(Red = Most Critical: {most_critical})")

# Disrupted Network
plt.subplot(1, 2, 2)
# Reuse same positions (skip missing node)
pos_disrupted = {n: pos[n] for n in G_disrupted.nodes()}

node_colors_disrupted = []
for node in G_disrupted.nodes():
    if node in orphaned_after:
        node_colors_disrupted.append('orange')      # lost all supply
    elif G_disrupted.out_degree(node) == 0 and G_disrupted.in_degree(node) > 0:
        node_colors_disrupted.append('lightgreen')  # end customer (retailer)
    else:
        node_colors_disrupted.append('lightblue')

nx.draw(
    G_disrupted, pos_disrupted,
    node_color=node_colors_disrupted,
    node_size=900,
    with_labels=True,
    font_size=8,
    font_weight='bold',
    arrows=True,
    arrowsize=12,
    edge_color='gray',
    width=1.0
)

# Add legend manually
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='red', label='Removed Node'),
    Patch(facecolor='orange', label='Orphaned (No Input)'),
    Patch(facecolor='lightgreen', label='End Customer'),
    Patch(facecolor='lightblue', label='Operational')
]
plt.legend(handles=legend_elements, loc='upper right')

plt.title(f"After Removing {most_critical}\n(Orange = Orphaned Nodes)")
plt.tight_layout()
plt.show()